# Momento de Retroalimentación: Reto Evaluación y Refinamiento de modelo

**TC3004B.104 Inteligencia Artificial Avanzada para la Ciencia de Datos**

Integrantes del equipo:
* Luis Ángel Guzmán Iribe - A01741757
* Julian Lawrence Gil Soares - A00832272
* Alberto H Orozco Ramos - A00831719

Profesores:
* Ivan Mauricio Amaya Contreras
* Hugo Terashima Marín
* Blanca Rosa Ruiz Hernandez
* Antonio Carlos Bento
* Frumencio Olivas Alvarez


## Introducción
Este documento es la última etapa para el proyecto final de la concentración de Inteligencia artificial avanzada para la ciencia de datos (TC3006C.101) del Tecnológico de Monterrey, la cual sirve como curso introductorio a temas de analitica de datos, construcción y uso de modelos de machine learning y evaluación de los mismos.

Previamente se trabajó en la limpieza de un dataset, en la implementación de diferentes tecnicas de machine learning y la evaluación de las mismas para  

### Reto

Como proyecto final del curso, se trabaja sobre la competencia de Kaggle [Titanic - Machine Learning from Disaster](https://www.kaggle.com/competitions/titanic), la cual es conocida en circulos de machine learning como un ejercicio clasico en la introducción al tema. El reto consiste en desarrollar un modelo de machine learning capaz de predecir si un determinado pasajero del Titanic sobrevivió o falleció en el undimiento del mismo tomando en cuenta información como su sexo, edad del pasajero, número de familiares a bordo, entre otras variables.

La competencia tiene como objetivo introducir a los participantes al desarrollo de modelos de machine learning por medio de la experimentación con diferentes algoritmos buscando encontrar la solución más adecuada para el reto, en el contexto del curso, sirve como una aplicación práctica de los temas vistos en clase.

### Dataset
En este proyecto se trabaja con el [dataset](https://www.kaggle.com/competitions/titanic/data) de pasajeros del Titanic, este dataset contiene las variables: *survival*, *pclass*, *sex*, *Age*, *sibsp*, *parch*, *ticket*, *fare*, *cabin* y *embarked*; puede leer más a detalle sobre el significado de cada variable en el dataset en la ligra proporcionada anteriormente, pero a modo de resumen, estas variables representan las caracteristicas de los pasajeros embarcados en el Titanic, su clase, sexo, edad, numero de familiares a bordo, el precio de su boleto, etc, meintras que la variable *survided* indica con un 0 o 1 si el pasajero sobrevivió. Es pues el objetivo del reto desarrollar un modelo que tome en cuenta las caracteristicas del pasajero para predecir el valor de la variable survived.

En iteraciones anteriores del trabajo se realizzó la limpieza del dataset, en la que se explica en detalle el proceso que seguimos para decidir que variables incluir en nuestro analisis, como tratar datos atipicos, valores faltantes, etc. Puede consultar a detalle el trabajo realizado y el resultado en [este archivo](https://github.com/4lb3rt0r/TC3006_Equipo2/blob/main/retro/Limpieza%20de%20Datos/clean_training_data.ipynb).

En el archivo ```train_clean.csv``` se encuentra una versión procesada del dataset original, en la que se han extraído las variables que consideramos últiles para nuestro analisis (*Survived*, *Pclass*, *Age*, *SibSp*, *Parch* y *Fare*), se tratado los valores nulos, se han convirtido variables categorigas como *cabin* y *sex* en variables dummy numericas *hasCabin* y *isFemale*.

**Lectura de ```train_clean.csv```**

In [1]:
import pandas as pd

# Si se trabaja en Colab
from google.colab import drive
drive.mount('/content/drive')
trainig_data = pd.read_csv('/content/drive/MyDrive/Equipo 2 - TC3006/Reto - Algoritmos ML/train_clean.csv')

# Si se trabaja en local
# trainig_data = pd.read_csv('train_clean.csv')

Mounted at /content/drive


En esta sección de código se extrae el contenido de variables guardadas en el dataframe hacia arreglos de la librería *numpy*, en el arreglo ``target`` se almacena la variable objetivo *Survived*, es decir, la que nos intereza ser capaces de predecir; mientras que en ``features`` se almacenan las variables de entrada que usaremos para el entrenamiento del modelo. *Pclass, Age, SibSp, Parch, Fare, HasCabin y IsFemale*

En adición a esto, se dividen los datos en 2 conjuntos, uno de entrenamiento que se compone del 80% del conjunto original de los datos y se almacena en las variables ``train_features`` y ``train_targets``, y cun conjunto de datos de prueba que se compone del 20% de los valores originales para probar el desempeño de los modelos, se almacenan en las variables ``test_features`` y ``test_targets``.

Ambos conjuntos se seleccionan de manera aleatoria entre los datos, pero se selecciona una semilla (en este caso *42*) con la finalidad de que los datos sean reproducibles al volver a correr al código. Se utilizan los mismos conjuntos de entrenamiento y prueba para todos los modelos.

**Diivisión en subconjuntos de entrenamiento y pruebas**

In [2]:
# Importamos numpy y SciKitLearn para extraer y seccionar los datos
import numpy as np
from sklearn.model_selection import train_test_split

# Guardamos los datos de entrada y excluimos los datos de salida
target = np.array(trainig_data['Survived'])
# Guardamos los datos de salida para posteriores evaluaciones del modelo
features = np.array(trainig_data.drop('Survived', axis = 1))

train_features, test_features, train_targets, test_targets = train_test_split(features, target, test_size = 0.20, random_state = 42)

# Importamos el resto de librerías necesarias para crear el modelo, implementar GridSearch y medir métricas de rendimiento
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import make_scorer, accuracy_score, accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
import os

## Modelos preeliminares

En la entrega pasada se probaron diferentes modelos de machine learning buscando el más apto para abordar la solución del reto. En total, se probaron 4 tipos de modelo diferentes: arboles de decición, bosques aleatorios, Redes neuronales y máquinas de vectores de soporte; para cada tipo de modelo, se entrenaron 4 modelos diferentes variando los hiperparametros buscando encontrar los más apropiados para cada uno.

Se evaluó el desempeño de los modelos generados usando una matriz de confusión, en la que se comparan los verdaderos positivos, verdadero negativo, falsos positivos y falsos negativos con la finalidad de extraer las metricas de rendimiento: exactitud, exhaustividad, precisión y F1. Se extrajo la mejor prueba de cada modelo con la finalidad de compararla contra los demás modelos tomando en cuenta el valor F1 para determinar el rendimiento general del modelo.

El valor F1 se define de la siguiente manera:

$F1 = 2 \cdot \frac{Precision \cdot Recall}{Precision + Recall}$

El valor F1 sirve como un equilibrio entre la presición y exhaustividad, además de que permite un mayor enfoque en los casos erroneos del problema. Decidimos emplearlo como metrica de desempeño ya que existe una gran disparidad entre nuestras categorías, dado que el número de fallecidos supera al de supervivientes 2 a 1, además de que suele recomendar su uso cuando las categorías tienen implicaciones o pesos diferentes, en este caso, consideramos que predecir si un pasajero vive o muere es una implicación con pesos diferentes.

Para una descripción detallada de los modelos y metologías empleadas, puede consultar [este archivo](https://github.com/4lb3rt0r/TC3006_Equipo2/tree/main/retro/Reto%20Selección%2C%20configuración%20y%20entrenamiento%20del%20modelo).

Los resultados obtenidos son los siguientes:

| Prueba                     | Desempeño (Puntaje F1)|
|----------------------------|-----------------------|
| Prueba Árbol de Decisión 3 | 0,7746 |
| Prueba Bosque Aleatorio 1  | 0,7619 |
| Prueba Redes Neuronales 2  | 0,7821 |
| Prueba SVM 1               | 0,7272 |

De estos modelos, observamos que las redesneuronales tuvieron el mejor desempeño tomando en cuenta el valor F1. Por este motivo, decidimos desarrollar más en profundidad el modelo, y tomarlo como base para el modelo refinado de la solución final del reto.

## Refinamiento del modelo

Habiendo definido el modelo definitivo a utilizar para predecir de forma certera los datos de los supervivientes del Titanic, decidimos implementar una técnica comúnmente utilizada para optimizar y refinar los hiperparámetros de los modelos de Machine Learning. El objetivo es obtener la mejor configuración posible para nuestro modelo de *redes neuronales*, que demostró ofrecer los resultados más prometedores en comparación con otros enfoques, como Decision Trees, Random Forests y Support Vector Machine.
Esta técnica de refinamiento se conoce como *Grid Search, una herramienta ampliamente empleada para mejorar modelos de Machine Learning. Grid Search no se limita exclusivamente a redes neuronales; también se aplica con éxito en otros tipos de modelos, como Decision Trees, Support Vector Machines, Random Forests, Gradient Boosting, entre otros. Grid Search tiene la capacidad de buscar y mejorar los hiperparámetros de un modelo, que pueden incluir elementos como *batch_size, learning_rate (alpha), profundidad en Decision Trees, cantidad de neuronas y capas en redes neuronales, tipos de kernels, y muchos más ejemplos. A continuación presentaremos la implementación de esta metodología dentro de nuestro modelo de Neural Networks:

#### Definición de modelo mediante función de TensorFlow
La definición del modelo se encuentra dentro de una función debido a que necesitamos probar el modelo con diversas configuraciones, esto con el fin de encontrar el que mejor resultados arroje:

In [3]:
def create_model(optimizer='adam', activation='relu', neurons1=50, neurons2=5, neurons3=10):
    caracteristic_number = train_features_scaled.shape[1]  # Calculate the number of input features
    model = keras.Sequential([
        keras.layers.Dense(neurons1, activation=activation, input_shape=(caracteristic_number,)),
        keras.layers.Dense(neurons2, activation=activation),
        keras.layers.Dense(neurons3, activation=activation),  # Adding another hidden layer
        keras.layers.Dense(10, activation='sigmoid')
    ])
    model.compile(optimizer=optimizer, loss='mean_squared_error', metrics=['accuracy'])
    return model

#### Normalización de Datos

In [4]:
scaler = StandardScaler()
train_features_scaled = scaler.fit_transform(train_features)
test_features_scaled = scaler.transform(test_features)

#### Definición de Red Neuronal
Creamos el modelo de TensorFlow como un estimador de scikitLearn

In [5]:
model = keras.wrappers.scikit_learn.KerasClassifier(build_fn=create_model, verbose=0)

<ipython-input-5-8dc1a02b4106>:1: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model = keras.wrappers.scikit_learn.KerasClassifier(build_fn=create_model, verbose=0)


#### Definimos el grid de parámetros o configuraciones a probar con el modelo

Para nuestro modelo, nos enfocamos en afinar los hiperparámetros relacionados con las funciones de activación, cantidad de capas, cantidad de neuronas y el optimizador, con el objetivo de lograr los mejores resultados de predicción posibles. Actualmente, nuestra meta es mejorar la exactitud del modelo, que actualmente se sitúa alrededor del 82%.

En el caso de nuestro Grid Search, se probarón diferentes optimizadores, como "adam" y "sgd", se alternarón entre las funciones de activación "relu" y "sigmoid", se exploró con una cantidad de neuronas entre 50, 40 y 20 para la primera capa, 15, 10 y 5 para la segunda y 2, 5 y 10 para la tercera, la última capa se quedará fija con una neurona y función de activación “sigmoid”. Se evaluó el valor de "epochs" entre 5, 10, 20 y 30 iteraciones, y el "batch_size" variará entre 16, 32 y 64, lo que nos permitió evaluar diversas configuraciones y seleccionar la más adecuada para nuestro modelo.

Para la primera prueba, definimos una configuración inicial del Grid Search como sigue:


In [6]:
param_grid = {
    'optimizer': ['adam', 'sgd'],
    'activation': ['relu', 'sigmoid', 'softmax', 'tanh'],
    'neurons1': [50, 40, 20],
    'neurons2': [15, 10, 5],
    'neurons3': [2, 5, 10],
    'epochs': [5, 10, 20, 30],
    'batch_size': [16, 32, 64]
}

#### Definimos una función de puntaje

In [7]:
scoring = {'accuracy': make_scorer(accuracy_score)}

#### Creamos el GridSearchCV

In [8]:
grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=3, scoring=scoring, refit='accuracy')

#### Aplicamos GridSearch
El código de configuración del Grid Search evaluará cada una de estas configuraciones, alternará entre las opciones proporcionadas y comparará los resultados. Este proceso se repetirá hasta encontrar el conjunto de hiperparámetros que se consideren más óptimos en función de los resultados generados. Implica probar múltiples combinaciones de hiperparámetros, entrenar el modelo con cada una de ellas, registrar los resultados y comparar el rendimiento de cada uno para finalmente seleccionar el mejor modelo.

In [ ]:
grid_result = grid.fit(train_features_scaled, train_targets)

8/8 [==============================] - 0s 2ms/step


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:794: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_scorer.py", line 115, in __call__
    score = scorer._score(cached_call, estimator, *args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_scorer.py", line 276, in _score
    y_pred = method_caller(estimator, "predict", X)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_scorer.py", line 73, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/keras/wrappers/scikit_learn.py", line 270, in predict
    return self.classes_[classes]
IndexError: index 6 is out of bounds for axis 0 with size 2

  warnings.warn(


8/8 [==============================] - 0s 2ms/step


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:794: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_scorer.py", line 115, in __call__
    score = scorer._score(cached_call, estimator, *args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_scorer.py", line 276, in _score
    y_pred = method_caller(estimator, "predict", X)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_scorer.py", line 73, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/keras/wrappers/scikit_learn.py", line 270, in predict
    return self.classes_[classes]
IndexError: index 5 is out of bounds for axis 0 with size 2

  warnings.warn(


8/8 [==============================] - 0s 2ms/step


...

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:794: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_scorer.py", line 115, in __call__
    score = scorer._score(cached_call, estimator, *args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_scorer.py", line 276, in _score
    y_pred = method_caller(estimator, "predict", X)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_scorer.py", line 73, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/keras/wrappers/scikit_learn.py", line 270, in predict
    return self.classes_[classes]
IndexError: index 4 is out of bounds for axis 0 with size 2

  warnings.warn(


8/8 [==============================] - 0s 2ms/step


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:794: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_scorer.py", line 115, in __call__
    score = scorer._score(cached_call, estimator, *args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_scorer.py", line 276, in _score
    y_pred = method_caller(estimator, "predict", X)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_scorer.py", line 73, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/keras/wrappers/scikit_learn.py", line 270, in predict
    return self.classes_[classes]
IndexError: index 3 is out of bounds for axis 0 with size 2

  warnings.warn(


8/8 [==============================] - 0s 2ms/step


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:794: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_scorer.py", line 115, in __call__
    score = scorer._score(cached_call, estimator, *args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_scorer.py", line 276, in _score
    y_pred = method_caller(estimator, "predict", X)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_scorer.py", line 73, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/keras/wrappers/scikit_learn.py", line 270, in predict
    return self.classes_[classes]
IndexError: index 2 is out of bounds for axis 0 with size 2

  warnings.warn(


#### Desplegamos los mejores hipeparámetros y resultados obtenidos con GridSearch

In [ ]:
best_score = grid_result.best_score_
best_params = grid_result.best_params_

print("Best: %f using %s" % (best_score, best_params))

#### Obtenemos el mejor modelo

In [ ]:
best_model = grid_result.best_estimator_

#### Realizamos predicciones en el set de prueba utilizando el mejor modelo

In [ ]:
predictions = np.round(best_model.predict(test_features_scaled))

#### Evaluación de Rendimiento del Modelo

In [ ]:
# Calculamos métricas de evaluación
accuracy = accuracy_score(test_targets, predictions)
precision = precision_score(test_targets, predictions)
recall = recall_score(test_targets, predictions)
f1 = f1_score(test_targets, predictions)

# Declaramos la matriz de confusión
conf_matrix = confusion_matrix(test_targets, predictions)

# Desplegamos los resultados
plt.figure(figsize=(8,6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', cbar=False,
            xticklabels=['Not Survived', 'Survided'],
            yticklabels=['Not Survided', 'Survided'])

plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

#### Registramos los resultados en un archivo de salida

In [ ]:
# Guardamos los mejores hiperparámetros y resultados de rendimiento dentro de un archivo txt
# ***Modificar de ser necesario***
output_path = "/content/drive/MyDrive/Equipo 2 - TC3006/Reto - Algoritmos ML"
output_file = os.path.join(output_path, "hyperparameter_results.txt")

if not os.path.isfile(output_file):
    with open(output_file, 'w') as f:
        f.write("Hyperparameter Search Results:\n")

with open(output_file, 'a') as f:
    f.write(f"\nBest Score: {best_score}\n")
    f.write(f"Best Parameters: {best_params}\n")
    f.write("Parameter Grid:\n")
    for key, value in param_grid.items():
        f.write(f"{key}: {value}\n")
    f.write(f"\nTest Accuracy: {accuracy * 100:.2f}%\n")

## Resultados

El algoritmo arrojó resultados algo peculiares, y es que al parecer está eligiendo una combinación de hiper-parámetros que no resultan ser muy distintos o mejores en comparación a la configuración que teníamos previamente en entregas anteriores. Además, el código para implementar GridSearch demora demasiado en ejecutarse, por lo que no nos fue posible probar con gran cantidad de combinaciones, más problemas de conexión o fallos del código posterior a la implementación del GridSearch. A pesar de ello, logramos probar algunas configuraciones, que podrán no ser las que mejor redimiento representan, pero son los resultados que nos proporcionó esta metodología hasta el momento:


| Configuración Original de la Red Neuronal        | Hyperparameter GridSearch Results              | Hyperparameter GridSearch Results              | Hyperparameter GridSearch Results              |
|--------------------------------------------------|--------------------------------------------------|--------------------------------------------------|--------------------------------------------------|
| Best Score: 0.8212290502793296                   | Best Score: 0.8160243000153647                   | Best Score: 0.8188313300003546                   | Best Score: 0.8230448297462445                   |
| Best Parameters:                                 | Best Parameters:                                 | Best Parameters:                                 | Best Parameters:                                 |
| - 'activation': 'relu'                          | - 'activation': 'relu'                          | - 'activation': 'relu'                          | - 'activation': 'relu'                          |
| - 'batch_size': 32                              | - 'batch_size': 64                              | - 'batch_size': 32                              | - 'batch_size': 16                              |
| - 'epochs': 50                                  | - 'epochs': 50                                  | - 'epochs': 30                                  | - 'epochs': 20                                  |
| - 'neurons1': 50                               | - 'neurons1': 50                               | - 'neurons1': 10                               | - 'neurons_per_layer': 20                      |
| - 'neurons2': 15                               | - 'neurons2': 15                               | - 'neurons2': 15                               | - 'num_layers': 2                              |
| - 'neurons3': 75                               | - 'neurons3': 75                               | - 'neurons3': 6                                | - 'optimizer': 'adam'                         |
| - 'optimizer': 'adam'                          | - 'optimizer': 'adam'                          | - 'optimizer': 'adam'                          | - 'activation': 'relu'                         |
| Parameter Grid:                                  | Parameter Grid:                                  | Parameter Grid:                                  | Parameter Grid:                                  |
| - optimizer: ['adam']                           | - optimizer: ['adam', 'sgd']                   | - optimizer: ['adam', 'sgd']                   | - optimizer: ['adam', 'sgd']                   |
| - activation: ['relu', 'sigmoid']               | - activation: ['relu', 'sigmoid']               | - activation: ['relu', 'sigmoid', 'softmax', 'tanh'] | - activation: ['relu', 'sigmoid', 'softmax', 'tanh'] |
| - neurons1: [50]                               | - neurons1: [50, 100]                         | - neurons1: [5, 10]                           | - num_layers: [1, 2, 3]                        |
| - neurons2: [5]                                | - neurons2: [30, 15]                          | - neurons2: [10, 15]                         | - neurons_per_layer: [5, 10, 15, 20]           |
| - neurons3: [10]                               | - neurons3: [75, 40]                          | - neurons3: [6, 12]                           | - epochs: [5, 10, 20, 30]                      |
| - epochs: [50]                                 | - epochs: [50, 100]                           | - epochs: [5, 10, 20, 30]                      | - batch_size: [16, 32, 64]                     |
| - batch_size: [32]                             | - batch_size: [32, 64]                         | - batch_size: [16, 32, 64]                     | Test Accuracy: 79.89%                           |
|--------------------------------------------------|--------------------------------------------------|--------------------------------------------------|--------------------------------------------------|
| Test Accuracy: 81.01%                           | Test Accuracy: 79.89%                           | Test Accuracy: 79.89%                           | Test Accuracy: 79.89%                           |


## Conclusiones

El procedimiento de Grid Search es una búsqueda exhaustiva que genera candidatos tentativos para los hiperparámetros que mejor se ajusten al modelo y a los datos de entrada, en este caso, el conjunto de datos del Titanic. Esto nos permite explorar diversas opciones y configuraciones para encontrar la que proporcione los resultados óptimos y se adapte mejor al contexto del problema. Decidimos que esta metodología es una de las mejores opciones que pudimos elegir para probar múltiples configuraciones, compararlas y clasificarlas de acuerdo a su rendimiento y mejoramiento de resultados en comparación con los obtenidos previamente en entregas pasadas, que es de 82% de accuracy aproximadamente. El factor que influencia bastante los resultados de esta entrega fue el tiempo, el método demora demasiado tiempo en ejecutarse y en probar las distintas opciones que le proporcionamos, es por ello que seguiremos intentando implementar GridSearch previo a la entrega final para obtener la configuración más óptima de nuestro modelo.

## Referencias

* 3.2. Tuning the hyper-parameters of an estimator. (s. f.). scikit-learn. https://scikit-learn.org/stable/modules/grid_search.html